In [1]:
# Display the directory from where we are getting the packages.
.libPaths()
# One or more of the commands below will fail if the library is not installed.
library(e1071)
library(caret)
library(class)
library(rpart)
#install.packages('e1071', dependencies=TRUE)
require('e1071')
#install.packages('caret', dependencies = TRUE)
require(caret)

[1] "C:/Users/radeshpa/AppData/Local/Programs/Python/Python37/Lib/site-packages/rpy2/R/win-library/3.5"
[2] "C:/Program Files/R/R-3.5.2/library"

Loading required package: lattice
Loading required package: ggplot2


In [11]:
# To measure time (for debugging)
start.time <- Sys.time()
geneExpressionFileName  <- 'gene_expression_n438x978.txt' 
myData <- read.table(geneExpressionFileName, sep="\t", header=T)
dimnames(myData)[[1]] <- myData[,1]
myData <- myData[,-1]
myData[1:5, 1:5]
end.time <- Sys.time()
cat("took time: ",end.time - start.time)
dim(myData)

,PSME1,ATF1,RHEB,FOXO3,RHOA
ACETAZOLAMIDE,-0.015159099,-0.031470528,-0.004733488,0.02591061,0.0056296773
IRBESARTAN,-0.026811981,0.012151979,-0.025550148,-0.02401181,-0.0106717396
IPRATROPIUM BROMIDE,0.001017958,-0.008650622,-0.018128698,-0.02079971,-0.0002722781
EFAVIRENZ,-0.004398264,0.055387992,0.004658520,0.01380732,-0.0340697348
THIAMINE,0.001838965,-0.018079188,-0.011855532,-0.03705033,-0.0133954959


took time:  0.7585461

[1] 438 978

In [12]:
sideEffectFileName <- 'ADRs_HLGT_n438x232.txt'
sideEffectData <- read.table(sideEffectFileName, sep="\t", header=T)
dimnames(sideEffectData)[[1]] <- sideEffectData[,1]
sideEffectData <- sideEffectData[,-1]
sideEffectData[1:5, 1:5]
names <- names(sideEffectData)
dim(sideEffectData)

,Abdominal.hernias.and.other.abdominal.wall.conditions,Abortions.and.stillbirth,Acid.base.disorders,Administration.site.reactions,Adrenal.gland.disorders
ACETAZOLAMIDE,0,0,1,1,0
IRBESARTAN,0,0,0,0,1
IPRATROPIUM BROMIDE,0,0,0,0,1
EFAVIRENZ,0,0,0,0,1
THIAMINE,0,0,0,0,0


[1] 438 232

In [13]:
start.time <- Sys.time()
sideEffectNames <- names(sideEffectData)
geneNames <- names(myData)
experimentName <- "t_0_03_kNN_k_10"
experimentResults <- data.frame(a = character(), b= numeric())
experimentResults <- rbind(experimentResults, data.frame(a = "side effect name", b = 0))

################################### EXPERIMENT OUTPUT FILE NAME ################################################################
fileName <- paste(experimentName, '.csv')
################################################################################################################################

experimentFile <- paste("C:\\Users\\radeshpa\\Desktop\\BioInformatics\\Assignment3\\Experiment_output", fileName)

for(i in 215:length(sideEffectNames)){
    sideEffectName <- sideEffectNames[i]
    cat("Working on the side effect: ", sideEffectName, "\n")
    currentSideEffect <- sideEffectData[,sideEffectName]
    presentIndices <- which(currentSideEffect == 1)
    absentIndices <- which(currentSideEffect == 0)
   
    outerCount <- 3
    accuracyOverFoldsTotal <- 0
    
    for(j in 1:outerCount){
        
        
################################################  DATA  #######################################################################
        
        foldCount <- 10
        folds <- createFolds(currentSideEffect, foldCount, returnTrain = FALSE)
        # Get the values for every list (indices)

        foldIndices <- list()
        
        for(k in 1:length(folds)){
            foldIndices[[length(foldIndices)+1]] <- folds[[k]]
        }

        accuracyOverFolds <- 0
        accuracyOverFoldsSum <- 0
       
        
        for(k in 1:length(folds)){
            # Data of this fold.
            cat("Running fold number: ", k, " Count number: ", j, "\n")

            # We need to create a vector for indexing
            testingDataX <- myData[foldIndices[[k]],]


            testingDataY <- sideEffectData[foldIndices[[k]],sideEffectName]


            remainingIndices <- (Reduce(c,foldIndices[-k]))

            trainingDataX <- myData[c(remainingIndices), ]

            trainingDataY <- sideEffectData[c(remainingIndices), sideEffectName]

##################################################### FEATURE SELECTION : t- statistic ###########################################################            

            allpValues <- sapply(geneNames, function(geneName){
                geneData <- trainingDataX[,geneName]
                sampleOne <-  geneData[presentIndices]
                sampleOneLength <- length(sampleOne)
                sampleTwo <-  geneData[absentIndices]
                sampleTwoLength <- length(sampleTwo)
                if(sampleOneLength == 0 | sampleTwoLength == 0 | (sampleOneLength == 1 && sampleTwoLength != 1) | (sampleOneLength != 1 && sampleTwoLength == 1)){
                    pValue  <- 0 # we want this feature.
                }
                else{
                    pValue <- t.test(sampleOne, sampleTwo)$p.value
                }
                pValue            
            })

            ######## THRESHOLD FOR THE STATISTIC #############
            
            threshold <- 0.03
            sigpIndices <- which(allpValues < threshold)

            sigpValues <- allpValues[sigpIndices]
            featuresOfInterest <- names(trainingDataX)[sigpIndices]

            refinedTrainingDataX <- trainingDataX[, featuresOfInterest]

            
########################################## KNN, K = 10 #######################################################################################

            # Now filter the test set also!
            refinedTestingDataX <- testingDataX[,featuresOfInterest]
            predictionResults <- knn(refinedTrainingDataX, refinedTestingDataX, factor(trainingDataY), k = 10)

            accuracy <- (sum(predictionResults==testingDataY) + sum(is.na(predictionResults) & is.na(testingDataY))) / length(predictionResults)
            accuracyOverFoldsSum <- accuracyOverFoldsSum + accuracy
     
#######################################################################################################################        

        }
        
        accuracyOverFolds <- (accuracyOverFoldsSum)/length(folds)     
        accuracyOverFoldsTotal <- accuracyOverFoldsTotal + accuracyOverFolds
    }
    
    finalAccuracy  <- accuracyOverFoldsTotal/outerCount
    cat("Final Accuracy is: ", finalAccuracy, "\n")
    
    write.table(data.frame(a = sideEffectName, b = finalAccuracy), experimentFile, append = TRUE, sep = "\t", row.names = FALSE, col.names = FALSE)    
}

end.time <- Sys.time()
cat("took time: ",end.time - start.time)

Working on the side effect:  Tongue.conditions 
Running fold number:  1  Count number:  1 
Running fold number:  2  Count number:  1 
Running fold number:  3  Count number:  1 
Running fold number:  4  Count number:  1 
Running fold number:  5  Count number:  1 
Running fold number:  6  Count number:  1 
Running fold number:  7  Count number:  1 
Running fold number:  8  Count number:  1 
Running fold number:  9  Count number:  1 
Running fold number:  10  Count number:  1 
Running fold number:  1  Count number:  2 
Running fold number:  2  Count number:  2 
Running fold number:  3  Count number:  2 
Running fold number:  4  Count number:  2 
Running fold number:  5  Count number:  2 
Running fold number:  6  Count number:  2 
Running fold number:  7  Count number:  2 
Running fold number:  8  Count number:  2 
Running fold number:  9  Count number:  2 
Running fold number:  10  Count number:  2 
Running fold number:  1  Count number:  3 
Running fold number:  2  Count number:  3 
Runn

took time:  2.212206